In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [2]:
# --- Configura o Selenium para rodar em modo "headless" (sem abrir janela)
chrome_options = Options()
chrome_options.add_argument("--headless")  # Remova essa linha se quiser ver o navegador
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# --- Inicializa o navegador
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
# --- URL do QR Code (exemplo real da nota)

"""
 O que significam os valores após a chave de acesso?
 p = CHAVE|VER|AMBIENTE|TIPO_EMISSOR|DIGEST
1. CHAVE:  Chave de acesso da NFC-e (29250306057223048450650150000775861151017568)
2. VERSÃO=2: Versão do QR Code (ex: 2 = layout da 2ª versão)
3. AMBIENTE=1: produção = 1, Homologação (teste) = 2
4. TIPO_EMISSOR=1:  geralmente 1 = normal, 2 = contingência
5. DIGEST/HASH: DigestValue (resumo criptográfico SHA1 ou SHA256)

O DigestValue é uma assinatura que autentica a URL (como um "hash de verificação" da NFC-e), criado com base nos dados da nota e certificado digital.
 Ele evita alterações manuais ou falsificações na URL.

 """

url_qrcode = "http://nfe.sefaz.ba.gov.br/servicos/nfce/modulos/geral/NFCEC_consulta_chave_acesso.aspx?p=29250306057223048450650150000775861151017568|2|1|1|8DF25CB77517F73F80677AE283AEE7DFCCA38282"
driver.get(url_qrcode)

In [4]:
# --- Aguarda a página carregar completamente
time.sleep(5)  # Pode ajustar dependendo da conexão

# --- Extrai o HTML renderizado
html = driver.page_source
driver.quit()

In [5]:
# --- Faz o parsing com BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

In [6]:
# --- Localiza a tabela com id="tabResult"
tabela = soup.find("table", {"id": "tabResult"})

In [ ]:
produtos = []

if tabela:
    linhas = tabela.find_all("tr")
    for linha in linhas:
        colunas = linha.find_all("td")
        if len(colunas) >= 2:
            # Descrição
            descricao_tag = colunas[0].find("span", class_="txtTit")
            descricao = descricao_tag.text.strip() if descricao_tag else ""

            # Código
            codigo_tag = colunas[0].find("span", class_="RCod")
            codigo = ""
            if codigo_tag:
                match = re.search(r"\d+", codigo_tag.text)
                if match:
                    codigo = match.group()

            # Extrai o restante do texto do td esquerdo
            info_extra = colunas[0].text

            # Regex para quantidade, unidade e valor unitário
            #qtd_match = re.search(r"Qtde:\s*([\d,\.]+)", info_extra)
            qtd_match = re.search(r"Qtde\.\s*:\s*([\d,\.]+)", info_extra)

            un_match = re.search(r"UN:\s*([A-Za-z]+)", info_extra)
            vl_unit_match = re.search(r"Vl\. Unit\.\s*:\s*([\d,]+)", info_extra)

            quantidade = qtd_match.group(1) if qtd_match else ""
            unidade = un_match.group(1) if un_match else ""
            valor_unitario = vl_unit_match.group(1) if vl_unit_match else ""

            # Valor total
            valor_tag = colunas[1].find("span", class_="valor")
            valor_total = valor_tag.text.strip() if valor_tag else ""

            produtos.append({
                "Descrição": descricao,
                "Código": codigo,
                "Quantidade": quantidade,
                "Unidade": unidade,
                "Valor Unitário": valor_unitario,
                "Valor Total": valor_total
            })

    df = pd.DataFrame(produtos)
    #print(df)
else:
    print("Tabela 'tabResult' não encontrada.")

In [22]:
df.head()

,Descrição,Código,Quantidade,Unidade,Valor Unitário,Valor Total
0,TOMATE SALADA kg,8186,"0,505",Kg,"7,49","3,78"
1,BETERRABA kg,8134,"0,215",Kg,"3,99","0,85"
2,CEBOLA NAC kg,7832,"0,33",Kg,"3,99","1,31"
3,BANANA TERRA kg,62885,"0,59",Kg,"3,49","2,05"
4,PITAYA kg,1062316,"0,48",Kg,"17,9","8,59"


In [24]:
df.to_csv("../data/raw/compras.csv")

In [ ]:
valor_total = df['Valor Total'].sum()
print(valor_total)

3,780,851,312,058,599,614,784,401,890,940,981,433,834,449,304,492,491,895,994,499,902,1610,9047,8016,9012,5411,2210,4010,8118,2677,8023,311,233,8730,986,2921,763,9825,085,637,0637,9016,4917,9959,0021,9015,486,9924,9014,7012,8021,9636,9842,902,097,894,8915,9016,6012,871,258,754,184,8926,903,498,858,905,294,194,892,8512,9013,9029,803,553,4523,694,883,7419,193,693,693,556,0510,602,0913,292,8511,901,694,985,814,5813,495,2910,4910,582,992,997,993,132,997,9920,905,6725,9019,49
